In [2]:
import pandas as pd
import numpy as np

In [3]:
## Q7 Petri Nets
def reset():
    global state, produced, consumed, missing, remaining
    global positions, transitions, initial, final
    positions = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0
    }
    transitions = {
        'a': {
            'in' : [1],
            'out': [2, 3]
        },
        'l': {
            'in' : [2],
            'out': [4]
        },
        'e': {
            'in' : [4,5],
            'out': [2,3]
        },
        'c': {
            'in' : [3],
            'out': [5]
        },
        'm': {
            'in' : [5],
            'out': [3]
        },
        'f': {
            'in' : [4,5],
            'out': [6]
        },
    }
    initial = [1]
    final = [6]

    state = []

    produced, consumed, missing = [], [], []
    remaining = None

In [4]:
def save_state():
    global state
    state.append(positions.copy())

def fire(transition):
    global produced, consumed, missing
    prod, cons, miss = 0, 0, 0
    for p in transitions[transition]['in']:
        if positions[p] > 0:
            positions[p] -= 1
        else:
            miss += 1
        cons += 1
    for p in transitions[transition]['out']:
        positions[p] += 1
        prod += 1
    produced.append(prod)
    consumed.append(cons)
    missing.append(miss)
    save_state()

def initalize():
    global produced, consumed, missing
    produced, consumed, missing = [0], [0], [0]
    for initial_position in initial:
        positions[initial_position] += 1
        produced[0] += 1
    save_state()

def is_final():
    for final_position in final:
        if positions[final_position] == 0:
            return False
    global produced, consumed, missing, remaining
    cons = 0
    for final_position in final:
        cons += positions[final_position]
        positions[final_position] = 0
    consumed.append(cons)
    produced.append(0)
    missing.append(0)
    remaining = sum(positions.values())
    save_state()
    return True



In [7]:
for trace in ["aclelcf","lmclcf"]:
    print("Trace",trace)
    reset()
    initalize()
    for i,transition in enumerate(trace):
        fire(transition)
        if is_final():
            print("Final state reached after {} steps".format(i+1))
            print("Trace reached: {}".format(trace[:i+1]))
            break
    display(pd.DataFrame([produced, consumed, missing], columns=["init"] + list(trace) + ["final"], index=["produced", "consumed", "missing"]))
    summary = {
        "produced": sum(produced),
        "consumed": sum(consumed),
        "missing": sum(missing),
        "remaining": remaining
    }
    summary["fitness"] = 0.5*(1-(summary["missing"]/summary["consumed"])) + 0.5*(1-(summary["remaining"]/summary["produced"]))
    display(pd.DataFrame(summary, index=["total"]))
    display(pd.DataFrame(state, columns=positions.keys()))

Trace aclelcf
Final state reached after 7 steps
Trace reached: aclelcf


,init,a,c,l,e,l,c,f,final
produced,1,2,1,1,2,1,1,1,0
consumed,0,1,1,1,2,1,1,2,1
missing,0,0,0,0,0,0,0,0,0


,produced,consumed,missing,remaining,fitness
total,10,10,0,0,1.0


,1,2,3,4,5,6
0,1,0,0,0,0,0
1,0,1,1,0,0,0
2,0,1,0,0,1,0
3,0,0,0,1,1,0
4,0,1,1,0,0,0
5,0,0,1,1,0,0
6,0,0,0,1,1,0
7,0,0,0,0,0,1
8,0,0,0,0,0,0


Trace lmclcf
Final state reached after 6 steps
Trace reached: lmclcf


,init,l,m,c,l,c,f,final
produced,1,1,1,1,1,1,1,0
consumed,0,1,1,1,1,1,2,1
missing,0,1,1,0,1,1,0,0


,produced,consumed,missing,remaining,fitness
total,7,8,4,3,0.535714


,1,2,3,4,5,6
0,1,0,0,0,0,0
1,1,0,0,1,0,0
2,1,0,1,1,0,0
3,1,0,0,1,1,0
4,1,0,0,2,1,0
5,1,0,0,2,2,0
6,1,0,0,1,1,1
7,1,0,0,1,1,0


In [6]:
## (b)
cell_df = pd.DataFrame(
    np.array(
    [
        [10,6,9],
        [10,7,9],
        [0,2,1],
        [0,1,1]
    ]).T,
    columns=["produced", "consumed", "missing", "remaining"],
    index=['sigma_3', 'sigma_4', 'sigma_5']
)
cell_df.index.name = 'trace'

cell_df['fitness'] = 0.5*(1-(cell_df['missing']/cell_df['consumed'])) + 0.5*(1-(cell_df['remaining']/cell_df['produced']))
cell_df['n_runs'] = [40,30,30]
print("Total fitness: {}".format((cell_df['fitness'] * cell_df['n_runs']).sum()/cell_df['n_runs'].sum()))
display(cell_df)

Total fitness: 0.8988095238095238


,produced,consumed,missing,remaining,fitness,n_runs
trace,,,,,,
sigma_3,10,10,0,0,1.000000,40
sigma_4,6,7,2,1,0.773810,30
sigma_5,9,9,1,1,0.888889,30


### (d)
Sequence cut. Activity partitioning after cut: $\{a,c,m,l\},\{e\}$

### (e)
NO.

If someone feels like explaining this in the forum post, please do.